In [ ]:
import pandas as pd
import altair as alt
from pathlib import Path
# alt.renderers.enable("mimetype")
alt.renderers.enable('default')


df = pd.read_csv("https://raw.githubusercontent.com/onlyphantom/miband/main/data/run_1km.csv",
        parse_dates=['startTime', 'date'])
df['day_of_week'] = df.startTime.dt.day_name()


In [ ]:
df['week'] = df.startTime.dt.isocalendar().week
df.head(2)

In [ ]:
# %%time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

alt.Chart(df).mark_point().encode(
    # x = 'seconds_per_km:Q',
    # y = 'day_of_week:N',
    # color = 'day_of_week:N'
    alt.X('seconds_per_km',),
    alt.Y('day_of_week'),
    alt.Color('day_of_week')
    ).interactive()

In [ ]:
alt.Chart(df).mark_point(color='green').encode(
    alt.X('seconds_per_km', "min"),
    alt.Y('day_of_week'),
    # alt.Color('day_of_week'),
)

In [ ]:
alt.Chart(df).mark_point(color='#5f9ea0').encode(
    alt.X('seconds_per_km', "min", title='Seconds/km (min)', scale=alt.Scale(type='log')),
    alt.Y('day_of_week',  title='Day'),
)

In [ ]:
alt.Chart(df).mark_circle(color='#5f9ea0').encode(
    alt.X('seconds_per_km', "min", title='Seconds/km (min)', scale=alt.Scale(type='log')),
    alt.Y('day_of_week',  title='Day'),
)

In [ ]:
line = alt.Chart(df).mark_line(color='lightpink').encode(
    alt.X('week', title='Week number'),
    alt.Y('seconds_per_km', aggregate='mean',  title='Speed'),
)

circle = alt.Chart(df).mark_circle(color='deeppink').encode(
    alt.X('week', title='Week number'),
    alt.Y('seconds_per_km', aggregate='mean',  title='Speed'),
)
line + circle

In [ ]:
# Another way of doing the above is to set up a base chart and add more layers, like so

base = alt.Chart(df).mark_line(color='lightpink').encode(
    alt.X('week', title='Week number'),
    alt.Y('seconds_per_km', aggregate='mean',  title='Speed'),
)

base + base.mark_circle(color='magenta')

In [ ]:
# the same as above, but with `point=True`

alt.Chart(df).mark_line(color='lightpink', point=True).encode(
    alt.X('week', title='Week number'),
    alt.Y('seconds_per_km', aggregate='mean',  title='Speed'),
)


In [ ]:
# two charts in line (side by side)
speed = alt.Chart(df).mark_line(color='#A9A9A9', point=True).encode(
    alt.X('week', title='Week Number'),
    alt.Y("seconds_per_km", aggregate='mean')
    )

dist = alt.Chart(df).mark_line(color='#A9A9A9', point=True).encode(
    alt.X('week', title='Week Number'),
    alt.Y("distance(m)", aggregate='sum')
    )

speed | dist

In [ ]:
# another way to do side by side
speed = alt.Chart(df).mark_line(color='#A9A9A9', point=True).encode(
    alt.X('week', title='Week Number'),
    alt.Y("seconds_per_km", aggregate='mean')
    )

cal = speed.encode(
    alt.Y("calories(kcal)", aggregate='sum', title='Calories')
)

speed | cal.interactive() # `|` for side by side
# speed & cal # `&` for one above the other
# speed + cal # `&` to overlay different charts on the same graph


In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

speed = alt.Chart(df).mark_line(color='#A9A9A9', point=True).encode(
    alt.X('week', title='Week Number'),
    alt.Y("seconds_per_km", aggregate='mean'),
    tooltip=['date', 'distance(m)', 'maxPace(/meter)', 'seconds_per_km' ]
    )
speed.interactive()

In [ ]:

speed = alt.Chart(df).mark_point(color='#A9A9A9').encode(
    alt.X('week', title='Week Number'),
    alt.Y("seconds_per_km", aggregate='mean'),
    tooltip=['date', 'distance(m)', 'maxPace(/meter)', 'seconds_per_km' ]
    )
speed.interactive()

In [ ]:
alt.Chart(df).mark_bar().encode(
    alt.X('speed_per_km',  aggregate='count', title='Run count'),
    alt.Y("startTime:O", timeUnit='month'),
    
    )


In [ ]:
alt.Chart(df).mark_bar().encode(
    alt.X('speed_per_km',  aggregate='count', title='Run count'),
    alt.Y('hours(startTime):O')
    
    )

In [ ]:
alt.Chart(df).mark_bar().encode(
    alt.X('speed_per_km',  aggregate='count', title='Run count'),
    alt.Y('hours(startTime):T')

    )

In [ ]:
alt.Chart(df).mark_bar().encode(
    alt.X('hours(startTime):N'),
    alt.Y('speed_per_km:Q',  aggregate='count', title='Run count'),

    )

In [ ]:
%%time
# Heatmap

alt.Chart(df).mark_rect().encode(
    alt.X('date(startTime):O', title='day'),
    alt.Y('month(startTime):O', title='month'),
    color='max(distance(m)):Q'
    ).properties(
        title='Daily runs'
    )

In [ ]:
# create a selection / filter over y-axix encoding that changes color 
brush = alt.selection_interval(encodings=['y'])

# variable opacity based on if something is selected or not
opacity = alt.condition(brush, alt.value(1), alt.value(0.2))

runs = alt.Chart(df).mark_bar(color='lightpink').encode(
    alt.X('count(speed_per_km):Q', title='Run count'),
    alt.Y('month(startTime):O', title=None, timeUnit='month'),
    # tooltip=['max(speed_per_km)', 'count(speed_per_km)'],
    tooltip=[
        alt.Tooltip('max(speed_per_km)', title='max speed'),
        alt.Tooltip('count(speed_per_km):Q', title='# of runs')
    ],
    # opacity=alt.value(0.5) # manual opacity
    opacity=opacity # opacity based on if data is selected by mouse or not
    ).add_selection(brush).properties(
        title='Monthly Runs',
        height=240,
        width=300
    )

speed = alt.Chart(df).mark_circle(color='turquoise').encode(
    alt.X('week', title='Week Number'),
    alt.Y('seconds_per_km:Q'),
    opacity=opacity
    ).properties(
        title='Weekly Speed',
        height=240,
        width=300
    )

speed |runs 

In [ ]:

speed2 = alt.Chart(df).mark_circle(color='plum').encode(
    alt.X('week', title='Week Number'),
    alt.Y('seconds_per_km'),
    opacity=opacity
    ).add_selection(brush)

# print(speed2.to_json()[-400:])
# the same as above, but in javascrip notation

x = alt.X(field="week",title= "Week Number", type= "quantitative")

speed2

In [ ]:
x = alt.X('count(week):O')
print(x.to_json())


x = alt.X(aggregate='count', field="week", type= "ordinal")
print(x.to_json())

In [ ]:
speed2.save('speed2.html')